# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [15]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
# from IPython.display import Markdown, display
from IPython.display import Markdown, display, update_display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'


OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
# MODEL = "llama3.2"

In [3]:
# set up environment

# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

# Let's just make sure the model is loaded

!ollama pull llama3.2


API key found and looks good so far!


pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest 
pulling dde5aa3fc5ff... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 2.0 GB                         
pulling 966de95ca8a6... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.4 KB                         
pulling fcc5a6bec9da... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 7.7 KB                         
pulling a70ff7e570d9... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 6.0 KB                         
pulling 56bb8bd477a5... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�   96 B                         
pulling 34bb5ab01051... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [4]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

In [5]:
# here is the question; type over this to ask something new
system_prompt ="""
You are a patient tutor for a beginner programmer.  Please use multiple examples and analogies to explain the answer.
"""
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [6]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": question}
]

In [8]:
payload = {
        "model": MODEL_LLAMA,
        "messages": messages,
        "stream": False
    }

In [13]:
# Get gpt-4o-mini to answer, with streaming

def stream_response(system_prompt, question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [16]:
stream_response(system_prompt, question)

Sure! Let's break down this line of code step by step so you understand what it does and why it is written that way.

### Code Breakdown
1. **Set Comprehension**:
   - The expression `{book.get("author") for book in books if book.get("author")}` is a **set comprehension**. This is similar to list comprehensions but is used to create a set, which automatically removes duplicate values.
   - The `for book in books` part means we are looping through each `book` in a collection called `books`.
   - `book.get("author")` fetches the value associated with the key `"author"` from the `book` dictionary.
   - The `if book.get("author")` part serves as a filter. It checks whether the `author` exists and is not `None` (or otherwise "falsy"). If it does, then we get that author's name; if not, we skip to the next book.

   **Example**: Let's say you have a list of books like this:
   python
   books = [
       {"title": "Book A", "author": "Author A"},
       {"title": "Book B"},
       {"title": "Book C", "author": "Author B"},
       {"title": "Book D", "author": "Author A"},
   ]
   
   - The code generates a set of authors:
   python
   {'Author A', 'Author B'}
   
   This happens because `Author A` appears twice but is only listed once in the set.

2. **Yield From**:
   - The `yield from` keyword is used to delegate part of a generator’s operations to another generator. This means that instead of returning a single value, it allows a generator to yield multiple values from another iterable.
   - In this case, `yield from` will yield each author from the previously created set, one at a time.

### Putting it Together
So, the whole line of code essentially:
- Creates a set of unique author names from a list of book dictionaries, filtering out any books that do not have an author.
- Yields each unique author name one at a time.

### Analogy
If we think of this in more relatable terms:
- Imagine you're at a library and you are asked to list down all the unique authors for the books present.
- You walk through each book (the list of dictionaries) and write down the author if the book has one (the filter).
- As you write down the names, you notice you already wrote "Author A" recently, so you skip writing it again (the set, which avoids duplicates).
- In the end, instead of handing over the entire list at once, you choose to call out each name one by one (using `yield from`).

### Summary
In short, this line of code is useful for gathering a unique list of authors from a collection of book records and yields them one at a time for further processing. If you have any more questions or need further clarification, feel free to ask!

In [10]:
# Get Llama 3.2 # If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

This line of code is written in Python and utilizes a feature called "yield from" which allows the function to delegate iteration to another iterable.

Here's a breakdown:

- `for book in books`: This part iterates over each item (book) in the `books` collection.

- `.get("author") for book in books if book.get("author")`: For each book, this part tries to get the value of the key `"author"`. If the key doesn't exist (`None` is returned), it skips that book. This creates a generator expression which produces an iterable sequence.

- `yield from ...`: The `yield from` statement takes the sequence produced by the generator expression and yields each item directly, without creating a new container to hold them all.

So overall, this line of code will produce a sequence of books where the author is not missing (`None`), one book at a time, as if it came from the original iterable.

The benefits are:

1. More memory-efficient: Because we're not storing the entire sequence in memory at once,